Assuming fixed in and out group probailities, can we predict multiscale mixing based on group size?

For each class, track fraction of dataset and batch_id.  We expect the result to be a quartic equation based on all class sizes.

Wait--- can we predict just egh($\ell$) based on class sizes?  Once we have this the rest is easy ish I think.

In [1]:
from datetime import datetime
from lassort import load, localAssortF
import networkx as nx
import numpy as np
import pandas as pd
import scipy.sparse as sparse

from networkx.generators.community import stochastic_block_model as sbm
from lassort.localassort import calculateRWRrange
from networkx.algorithms.community import modularity
from matplotlib import pyplot as plt

In [228]:
seed = 777777
sizes = [10]*10
sizes.append(sum(sizes))

thresholds = np.zeros(len(sizes) + 1)
thresholds[1:] = sizes

for i in range(2, len(thresholds)):
    thresholds[i:] += sizes[i-2]

thresholds = thresholds.astype(int)

C = len(sizes)
p = np.ones((C,C)) * 0.05
for i in range(C):
    p[i,i] = 0.2
    
G = sbm(sizes, p.tolist(), seed=seed)

E = nx.convert_matrix.to_pandas_edgelist(G).values
M = np.array([y["block"] for x,y in G.nodes(data=True)]).astype(int)

# why is the OG score so okay?  Why aren't GF and GM being diluted into disassortativity?
A = nx.convert_matrix.to_scipy_sparse_matrix(G)
edgelist = E
node_attr = M
missingValue = -1

# number of nodes
n = len(node_attr)
# number od nodes with complete attribute
ncomp = (node_attr != missingValue).sum()
# number of edges
m = len(edgelist)
degree = np.array(A.sum(1)).flatten()

D = sparse.diags(1./degree, 0, format='csc')
W = D @ A # every row sums  to 1

c = len(np.unique(node_attr))

# calculate node weights for how "complete" the
# metadata is around the node
Z = np.zeros(n)
Z[node_attr == missingValue] = 1.
Z = (W @ Z) / degree

# indicator array if node has attribute data (or missing)
hasAttribute = node_attr != missingValue

# calculate global expected values
values = np.ones(ncomp)
yi = (hasAttribute).nonzero()[0]
yj = node_attr[hasAttribute]
Y = sparse.coo_matrix((values, (yi, yj)), shape=(n, c)).tocsc()
eij_glob = np.array(Y.T @ (A @ Y).todense())
eij_glob /= np.sum(eij_glob)
ab_glob = np.sum(eij_glob.sum(1)*eij_glob.sum(0)) # this is ar^2 for undirected

pr=np.arange(0., 1., 0.1)

assortM = np.empty((n, len(pr)))
assortT = np.empty(n)
WY = (W @ Y).tocsc()

In [258]:
pis, ti, it = calculateRWRrange(W, 150, pr, n)
pis.shape

(200, 10)

In [262]:
pis[:, 3][:100]

array([1.22568774e-04, 4.02089997e-04, 2.48936550e-04, 2.73947306e-04,
       5.56039603e-04, 3.99285281e-04, 2.11436194e-04, 1.91700967e-04,
       2.68712941e-04, 3.05408640e-04, 3.83601157e-04, 1.85215282e-04,
       1.51263188e-04, 1.49628296e-04, 3.91927992e-04, 2.50709717e-04,
       1.28265924e-04, 1.35390565e-04, 2.33460382e-04, 3.39488290e-04,
       7.67032961e-05, 4.12301227e-04, 5.70703920e-04, 3.11948908e-04,
       3.05287966e-04, 8.65323517e-05, 1.64265072e-04, 3.32993183e-04,
       4.18370567e-04, 2.48767626e-04, 4.55660225e-04, 1.90932403e-04,
       7.92684598e-05, 6.39293058e-04, 3.69042079e-04, 6.32372213e-04,
       8.25755765e-03, 4.94658103e-04, 7.46878351e-04, 1.78900104e-04,
       8.32194944e-03, 1.81356422e-04, 3.92446456e-05, 1.71092633e-04,
       2.84874422e-04, 3.49308864e-04, 4.44552755e-05, 2.79980585e-04,
       2.10908131e-04, 3.28983161e-04, 5.27620513e-05, 1.78392097e-04,
       8.35657419e-03, 8.25305816e-03, 6.57751820e-04, 1.96582914e-04,
      

In [35]:
(pis[0]).mean()

0.00037040958210002723

In [36]:
ti[0]

0.000423194203233852

In [37]:
(pis[10]).mean()

0.0014940390109100458

In [39]:
ti[10]

0.0017304212337004262

In [40]:
(pis[100]).mean()

0.002179727747765303

In [41]:
ti[100]

0.0025879116969980986

In [49]:
pis[-1].mean()

0.5557466998590959

In [53]:
pis[-1,1:].mean()

0.506385222065662

In [51]:
ti[-1]

0.5062048151528157

In [55]:
i

199

In [56]:
pis.mean(axis=1) - ti

array([-5.27846211e-05, -1.83602242e-04, -1.85142116e-04, -1.44401506e-04,
       -6.75806138e-05, -2.64058334e-04, -2.13671211e-04, -1.04502669e-04,
       -1.09478102e-04, -1.53029932e-04, -2.36382223e-04, -2.15176607e-04,
       -1.02435527e-04, -1.20815031e-04, -1.30617649e-04, -1.45736330e-04,
       -2.48931782e-04, -9.18018588e-05, -1.02780012e-04, -1.44389081e-04,
       -1.43645853e-04, -1.43335652e-04, -2.38797496e-04, -1.69772385e-04,
       -1.85871007e-04, -1.42512459e-04, -1.05020235e-04, -2.04807051e-04,
       -1.68975594e-04, -1.66820780e-04, -2.89642745e-04, -1.57554175e-04,
       -1.44685795e-04, -1.43771500e-04, -1.16626705e-04, -2.43509590e-04,
       -1.42018683e-04, -1.29484493e-04, -2.33916879e-04, -1.58342023e-04,
       -1.29600073e-04, -1.72943763e-04, -6.42194340e-05, -1.56065500e-04,
       -9.14732964e-05, -8.95504399e-05, -9.00061362e-05, -1.31252463e-04,
       -1.59085944e-04, -1.91183530e-04, -8.84275344e-05, -6.36172290e-05,
       -2.16077009e-04, -

Seems like the mean is a good enough approximation for everything EXCEPT node i itself.  We can easily get an expectation for totalRank if we know the pagerank values.

In [57]:
p

array([[0.2 , 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
       [0.05, 0.2 , 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.2 , 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.2 , 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.05, 0.2 , 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.05, 0.05, 0.2 , 0.05, 0.05, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.2 , 0.05, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.2 , 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.2 , 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.2 , 0.05],
       [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.2 ]])

# Test out approximtation of PPR
from this paper: 
https://arxiv.org/pdf/1910.12937.pdf

In [210]:
sizes = np.array(sizes)

# mixing matrix for DC-SBM from SBM
B = np.outer(sizes, sizes)

for i in range(C):
    s = sizes[i]
    B[i,i] = s * (s-1) * 2

B = B * p


# Expected total degree of each grp
theta = np.hstack([
    np.ones(sizes[i])*(1/sizes[i]) for i in range(len(sizes))])

# compute the diagonal degree matrix *D*
E = np.outer(sizes, sizes)

for i in range(C):
    s = sizes[i]
    E[i,i] = s * (s-1)

Dinv = np.diag(1/(E * p).sum(axis=1))

# block transition matrix
P = Dinv @ B
P

array([[0.31858407, 0.04424779, 0.04424779, 0.04424779, 0.04424779,
        0.04424779, 0.04424779, 0.04424779, 0.04424779, 0.04424779,
        0.44247788],
       [0.04424779, 0.31858407, 0.04424779, 0.04424779, 0.04424779,
        0.04424779, 0.04424779, 0.04424779, 0.04424779, 0.04424779,
        0.44247788],
       [0.04424779, 0.04424779, 0.31858407, 0.04424779, 0.04424779,
        0.04424779, 0.04424779, 0.04424779, 0.04424779, 0.04424779,
        0.44247788],
       [0.04424779, 0.04424779, 0.04424779, 0.31858407, 0.04424779,
        0.04424779, 0.04424779, 0.04424779, 0.04424779, 0.04424779,
        0.44247788],
       [0.04424779, 0.04424779, 0.04424779, 0.04424779, 0.31858407,
        0.04424779, 0.04424779, 0.04424779, 0.04424779, 0.04424779,
        0.44247788],
       [0.04424779, 0.04424779, 0.04424779, 0.04424779, 0.04424779,
        0.31858407, 0.04424779, 0.04424779, 0.04424779, 0.04424779,
        0.44247788],
       [0.04424779, 0.04424779, 0.04424779, 0.04424779, 0.

In [251]:
# Z is block membership matrix
Z = Y.todense()

# fix source node of PPR for now
u = 7
alpha = 0.3


pi = np.zeros(n)
pi[u] = 1

alpha_pi_t = pi @ Z * alpha

inv_transform = np.eye(C) - (1-alpha)*P
p = alpha_pi_t @ inv_transform

# blockwise PPR vector
p

matrix([[ 0.23309735, -0.00929204, -0.00929204, -0.00929204, -0.00929204,
         -0.00929204, -0.00929204, -0.00929204, -0.00929204, -0.00929204,
         -0.09292035]])

In [253]:
p / sizes[i]

matrix([[ 2.33097345e-03, -9.29203540e-05, -9.29203540e-05,
         -9.29203540e-05, -9.29203540e-05, -9.29203540e-05,
         -9.29203540e-05, -9.29203540e-05, -9.29203540e-05,
         -9.29203540e-05, -9.29203540e-04]])

In [265]:
pis[:, 3][10:20].sum()

0.002348950792826754

In [266]:
pis[:, 3][20:30].sum()

0.002927874116463403

In [267]:
pis[:, 3][30:40].sum()

0.012044562643869054

In [268]:
pis[:, 3][40:50].sum()

0.010212153583029342

In [269]:
pis[:, 3][50:60].sum()

0.018364056581048532

Mmmmmmm is this network just way too small for this to be a good approximation?  I think so.

Can we still use this algorithm to get an expectation for the multiscale mixing?  Possibly.

TODO: Figure out how to validate this.  Possibly just bigger SBM.

Next steps:

1. Integrate out alpha
2. Compute expected $e_{gh}(\ell)$ and mulitscale mixing.
3. Does this look better than individual PPR vectors?

In [198]:
A.sum()

3658

In [200]:
# I think this is all a mistake



array([ 113.,  113.,  113.,  113.,  113.,  113.,  113.,  113.,  113.,
        113., 2480.])